Exercise 2:

Python Version: 3.7
all newest versions of pytorch except for transforms.version=v1


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True, num_workers=2)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False, num_workers=2)

trainStep = len(train_loader.dataset) // 100
testStep = len(test_loader.dataset) // 100


# VGG11 model
class VGG11(nn.Module):
    def __init__(self):
        super(VGG11, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(512, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 10),
            nn.Softmax()
        )

    def forward(self, x):
        return self.cnn(x)




In [ ]:
print("[INFO] initializing CNN...")

# Instantiate the model
model = VGG11().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

stats = {
	"train_loss": [],
	"train_acc": [],
	"test_loss": [],
	"test_acc": []
}


In [ ]:
import time

print("[INFO] training the network...")
startTime = time.time()


# Training and Testing loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    trainLoss = 0
    trainCorrect = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        trainLoss += loss.item()
        trainCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

    doneTraining = time.time()
    print("[INFO] epoch time taken to train the model: {:.2f}s".format(doneTraining - startTime))




    # Evaluation
    model.eval()
    testLoss = 0
    testCorrect = 0

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)

            pred = model(x)
            loss = criterion(pred, y)

            testLoss += loss.item()
            testCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()


    # calculate the average training and validation loss
    avgTrainLoss = trainLoss / trainStep
    avgTestLoss = testLoss / testStep
    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(train_loader.dataset)
    testCorrect = testCorrect / len(test_loader.dataset)
    # update our training history
    stats["train_loss"].append(avgTrainLoss)
    stats["train_acc"].append(trainCorrect)
    stats["test_loss"].append(avgTestLoss)
    stats["test_acc"].append(testCorrect)

    print(f"Done Train and Test Epoch {epoch+1}/{num_epochs}")
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, trainCorrect))
    print("Test loss: {:.6f}, Test accuracy: {:.4f}\n".format(avgTestLoss, testCorrect))

endTime = time.time()
print("\n\n[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

print(stats)

torch.save(model, "trained_model.pt")




'train_loss': [1.5673993885517121, 1.4748803796370824, 1.471510537068049, 1.4692868530750274, 1.4681892536083858]

'train_acc': [0.9015833333333333, 0.9872, 0.9902666666666666, 0.9924833333333334, 0.9932666666666666]

'test_loss': [1.4757724356651307, 1.4715722870826722, 1.469182391166687, 1.4686590445041656, 1.4679065442085266]

'test_acc': [0.9867, 0.9901, 0.9924, 0.9929, 0.9933]

---

NOTE: pngs attatched seperately in submission



In [ ]:
import matplotlib.pyplot as plt

for key in stats:
  plt.figure()
  plt.plot(stats[key], label=key)
  plt.title(f"{key} on dataset")
  plt.xlabel("Epoch #")
  plt.ylabel("Loss/Accuracy")
  plt.savefig(key)


In [ ]:
#PART B)


# Data preprocessing
transformHorizontal = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
])


transformVertical = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
])

# MNIST dataset
htest_dataset = datasets.MNIST(root='./data', train=False, transform=transformHorizontal)
vtest_dataset = datasets.MNIST(root='./data', train=False, transform=transformVertical)

# Data loaders
htest_loader = DataLoader(dataset=htest_dataset, batch_size=100, shuffle=False, num_workers=2)
vtest_loader = DataLoader(dataset=vtest_dataset, batch_size=100, shuffle=False, num_workers=2)

htestStep = len(htest_loader.dataset) // 100
vtestStep = len(vtest_loader.dataset) // 100


# Evaluation
model.eval()
htestLoss = 0
htestCorrect = 0
vtestLoss = 0
vtestCorrect = 0

with torch.no_grad():
    for x, y in htest_loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = criterion(pred, y)

        htestLoss += loss.item()
        htestCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

    for x, y in vtest_loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = criterion(pred, y)

        vtestLoss += loss.item()
        vtestCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()


# calculate the average training and validation loss
avghTestLoss = htestLoss / htestStep
avgvTestLoss = vtestLoss / vtestStep
# calculate the training and validation accuracy
htestCorrect = htestCorrect / len(htest_loader.dataset)
vtestCorrect = vtestCorrect / len(vtest_loader.dataset)

bstats = {
	"htest_loss": [],
	"htest_acc": [],
	"vtest_loss": [],
	"vtest_acc": []
}

# update our training history
bstats["htest_loss"].append(avghTestLoss)
bstats["htest_acc"].append(htestCorrect)
bstats["vtest_loss"].append(avgvTestLoss)
bstats["vtest_acc"].append(vtestCorrect)


print(bstats)





'htest_loss': [0.34446121712525685], 'htest_acc': [0.3898]
'vtest_loss': [2.047345280647278], 'vtest_acc': [0.4096]

some numbers when flipped as we have can be viewed as the same number. For example horizontal flips on {8, 0, 1} could all be read as such when flipped horizontally, some on the vertical flip for numbers like {8, 0, 3}. So since we are getting accuracies of ~0.4 it seems reasonable that only a percentage of classes will remain interpretable after the transform.



In [ ]:
#PART Bii)

# Data preprocessing
transform1 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 0.01*torch.randn_like(x)),
])

transform2 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 0.1*torch.randn_like(x)),
])


transform3 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 1*torch.randn_like(x)),
])

# MNIST dataset
test1_dataset = datasets.MNIST(root='./data', train=False, transform=transform1)
test2_dataset = datasets.MNIST(root='./data', train=False, transform=transform2)
test3_dataset = datasets.MNIST(root='./data', train=False, transform=transform3)

# Data loaders
test1_loader = DataLoader(dataset=test1_dataset, batch_size=100, shuffle=False, num_workers=2)
test2_loader = DataLoader(dataset=test2_dataset, batch_size=100, shuffle=False, num_workers=2)
test3_loader = DataLoader(dataset=test3_dataset, batch_size=100, shuffle=False, num_workers=2)

test1Step = len(test1_loader.dataset) // 100
test2Step = len(test2_loader.dataset) // 100
test3Step = len(test3_loader.dataset) // 100

step = [test1Step, test2Step, test3Step]
sizes = [len(test1_loader.dataset), len(test2_loader.dataset), len(test3_loader.dataset)]

# Evaluation
model.eval()

testLoss = [0, 0, 0]
testCorrect = [0, 0, 0]

loaders = [test1_loader, test2_loader, test3_loader]

with torch.no_grad():

    for i in range(3):

        for x, y in loaders[i]:
            x, y = x.to(device), y.to(device)

            pred = model(x)
            loss = criterion(pred, y)

            testLoss[i] += loss.item()
            testCorrect[i] += (pred.argmax(1) == y).type(torch.float).sum().item()


#avgTestLoss = [testLoss[i]/step[i] for i in range(3)]

avgTestLoss = [testLoss[0]/step[0],testLoss[1]/step[1],testLoss[2]/step[2]]
avgTestCorrect = [testCorrect[0]/sizes[0],testCorrect[1]/sizes[1],testCorrect[2]/sizes[2]]


biistats = {
	"0.01test_loss": [],
	"0.01test_acc": [],
	"0.1test_loss": [],
	"0.1test_acc": [],
	"1test_loss": [],
	"1test_acc": []
}

# update our training history
biistats["0.01test_loss"].append(avgTestLoss[0])
biistats["0.01test_acc"].append(avgTestCorrect[0])
biistats["0.1test_loss"].append(avgTestLoss[1])
biistats["0.1test_acc"].append(avgTestCorrect[1])
biistats["1test_loss"].append(avgTestLoss[2])
biistats["1test_acc"].append(avgTestCorrect[2])


print(biistats)

**effect:** as we test on exponentially increasing variance - [0.01, 0.1, 1] we see steeper and steeper decline in accuracy [0.99, 0.96, 0.1] so the more gaussian noise the worse the accuracy

'0.01test_loss': [1.4728576004505158],
'0.01test_acc': [0.9882],

'0.1test_loss': [1.504005583524704],
'0.1test_acc': [0.9588],

'1test_loss': [2.362187247276306],
'1test_acc': [0.0979]




In [ ]:
#PART C

# Instantiate the model
model2 = VGG11().to(device)

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

transform1 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 0.01*torch.randn_like(x)),
])

transform2 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 0.1*torch.randn_like(x)),
])


transform3 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 1*torch.randn_like(x)),
])


mtransforms = [transform, transform1, transform2, transform3]

mdatasets = []
loaders = []

for i in range(len(mtransforms)):
    mdatasets.append(datasets.MNIST(root='./data', train=True, transform=mtransforms[i]))
    loaders.append(DataLoader(dataset=mdatasets[i], batch_size=100, shuffle=True, num_workers=2))


# Loss function and optimizer
crit = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.01)

num_epochs = 5

for epoch in range(num_epochs):
    model2.train()
    trainLoss = 0
    trainCorrect = 0

    for loader in loaders:
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            pred = model2(x)
            loss = crit(pred, y)

            opt.zero_grad()
            loss.backward()
            opt.step()


In [ ]:
#PART C TESTING)


# Data preprocessing
transformHorizontal = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.ToTensor(),
])


transformVertical = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
])

# MNIST dataset
htest_dataset = datasets.MNIST(root='./data', train=False, transform=transformHorizontal)
vtest_dataset = datasets.MNIST(root='./data', train=False, transform=transformVertical)

# Data loaders
htest_loader = DataLoader(dataset=htest_dataset, batch_size=100, shuffle=False, num_workers=2)
vtest_loader = DataLoader(dataset=vtest_dataset, batch_size=100, shuffle=False, num_workers=2)

htestStep = len(htest_loader.dataset) // 100
vtestStep = len(vtest_loader.dataset) // 100


# Evaluation
model.eval()
htestLoss = 0
htestCorrect = 0
vtestLoss = 0
vtestCorrect = 0

with torch.no_grad():
    for x, y in htest_loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = criterion(pred, y)

        htestLoss += loss.item()
        htestCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()

    for x, y in vtest_loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = criterion(pred, y)

        vtestLoss += loss.item()
        vtestCorrect += (pred.argmax(1) == y).type(torch.float).sum().item()


# calculate the average training and validation loss
avghTestLoss = htestLoss / htestStep
avgvTestLoss = vtestLoss / vtestStep
# calculate the training and validation accuracy
htestCorrect = htestCorrect / len(htest_loader.dataset)
vtestCorrect = vtestCorrect / len(vtest_loader.dataset)

bstats = {
	"htest_loss": [],
	"htest_acc": [],
	"vtest_loss": [],
	"vtest_acc": []
}

# update our training history
bstats["htest_loss"].append(avghTestLoss)
bstats["htest_acc"].append(htestCorrect)
bstats["vtest_loss"].append(avgvTestLoss)
bstats["vtest_acc"].append(vtestCorrect)


print(bstats)





#ii

#PART Cii)

# Data preprocessing
transform1 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 0.01*torch.randn_like(x)),
])

transform2 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 0.1*torch.randn_like(x)),
])


transform3 = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x : x + 1*torch.randn_like(x)),
])

# MNIST dataset
test1_dataset = datasets.MNIST(root='./data', train=False, transform=transform1)
test2_dataset = datasets.MNIST(root='./data', train=False, transform=transform2)
test3_dataset = datasets.MNIST(root='./data', train=False, transform=transform3)

# Data loaders
test1_loader = DataLoader(dataset=test1_dataset, batch_size=100, shuffle=False, num_workers=2)
test2_loader = DataLoader(dataset=test2_dataset, batch_size=100, shuffle=False, num_workers=2)
test3_loader = DataLoader(dataset=test3_dataset, batch_size=100, shuffle=False, num_workers=2)

test1Step = len(test1_loader.dataset) // 100
test2Step = len(test2_loader.dataset) // 100
test3Step = len(test3_loader.dataset) // 100

step = [test1Step, test2Step, test3Step]
sizes = [len(test1_loader.dataset), len(test2_loader.dataset), len(test3_loader.dataset)]

# Evaluation
model.eval()

testLoss = [0, 0, 0]
testCorrect = [0, 0, 0]

loaders = [test1_loader, test2_loader, test3_loader]

with torch.no_grad():

    for i in range(3):

        for x, y in loaders[i]:
            x, y = x.to(device), y.to(device)

            pred = model(x)
            loss = criterion(pred, y)

            testLoss[i] += loss.item()
            testCorrect[i] += (pred.argmax(1) == y).type(torch.float).sum().item()


#avgTestLoss = [testLoss[i]/step[i] for i in range(3)]

avgTestLoss = [testLoss[0]/step[0],testLoss[1]/step[1],testLoss[2]/step[2]]
avgTestCorrect = [testCorrect[0]/sizes[0],testCorrect[1]/sizes[1],testCorrect[2]/sizes[2]]


biistats = {
	"0.01test_loss": [],
	"0.01test_acc": [],
	"0.1test_loss": [],
	"0.1test_acc": [],
	"1test_loss": [],
	"1test_acc": []
}

# update our training history
biistats["0.01test_loss"].append(avgTestLoss[0])
biistats["0.01test_acc"].append(avgTestCorrect[0])
biistats["0.1test_loss"].append(avgTestLoss[1])
biistats["0.1test_acc"].append(avgTestCorrect[1])
biistats["1test_loss"].append(avgTestLoss[2])
biistats["1test_acc"].append(avgTestCorrect[2])


print(biistats)





'htest_loss': [2.0139254558086397],
'htest_acc': [0.4559],


'vtest_loss': [2.1255094814300537]
'vtest_acc': [0.3309]

'0.01test_loss': [1.5470370030403138]
'0.01test_acc': [0.9315]

'0.1test_loss': [1.5448921930789947]
'0.1test_acc': [0.9329]

'1test_loss': [1.6545295405387879]
'1test_acc': [0.8307]



---
We used a total of 6 training sets: {regular, horizontalFlip, verticalFlip, GausianNoise(GN)-0.01, GN-0.1, GN-1} in training the model with data augmentation
